In [ ]:
import torch
import clip
import laion_clap
import numpy as np
import librosa
from msclap import CLAP
#from transformers import ClapModel, ClapProcessor

In [ ]:
torch.cuda.is_available()

Load **CLIP**

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [33]:
model1, preprocess1 = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [03:19<00:00, 1.77MiB/s]


Load **Laion_CLAP**

In [ ]:
# quantization
def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)


def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    
#model2 = laion_clap.CLAP_Module(enable_fusion=False)
#model2.load_ckpt() # download the default pretrained checkpoint.
model2 = laion_clap.CLAP_Module(enable_fusion=False, amodel= 'HTSAT-base')
model2.load_ckpt('../Models/Laion_CLAP/music_audioset_epoch_15_esc_90.14.patched.pt')

load **Microsoft_CLAP**

In [ ]:
model3 = CLAP(version = '2023', use_cuda=True)

**CLIP** textdata-processing

In [ ]:
def process_and_add_embedding(csv_path, column_name, new_column_name):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "caption_clip" を追加
        df[new_column_name] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # ERROR回数カウント変数
        e_num = 0

        # テキストデータを順に処理して "clip_text_embed" に格納
        for i, text_data in enumerate(text_data_list):
            try:
                with torch.no_grad():
                    token = clip.tokenize(text_data).to(device)
                    embedding_tensor = model1.encode_text(token)

                # "clip_text_embed" 列にテンソルを格納
                df.at[i, new_column_name] = embedding_tensor

                # CUDAメモリを解放
                torch.cuda.empty_cache()

            except Exception as e:
                # エラーが発生した場合、例外メッセージを格納
                df.at[i, new_column_name] = "ERROR"
                print(str(e))
                e_num = e_num +1

        # 結果を表示
        print(df)
        print(f"ERROR数：{e_num}")

        # CSVファイルに結果を保存
        df.to_csv("../Datasets/archive/musiccaps-processed.csv", index=False)

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# CSVファイル、抽出列を指定
csv_file_path = "../Datasets/archive/musiccaps-public.csv"

target_column_name = "aspect_list" # clip_text_embed用
#target_column_name = "caption" # caption_clip用
create_column_name = "clip_text_embed"

process_and_add_embedding(csv_file_path, target_column_name, create_column_name)


**CLAP** textdata-processing

In [38]:
import pandas as pd

def clap_process_and_add_embedding(csv_path, column_name):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clap_text_embed" を追加
        df["clap_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # ERROR回数カウント変数
        e_num = 0

        # テキストデータを順に処理して "clap_text_embed" に格納
        for j, text_data in enumerate(text_data_list):
            try:
                # text_dataを2要素の配列に変換
                text_data_array = [text_data, ""]
                
                with torch.no_grad():
                    # model2.get_text_embeddingの前にtext_dataを配列に変換
                    text_embed = model2.get_text_embedding(text_data_array, use_tensor=True).to(device)

                    # [2, 512]のTensorから第一要素を取り出し、[1, 512]のTensorにする
                    text_embed = text_embed[0].unsqueeze(0)

                # "clap_text_embed" 列にテンソルを格納
                df.at[j, "clap_text_embed"] = text_embed

                # CUDAメモリを解放
                torch.cuda.empty_cache()

            except Exception as e:
                # エラーが発生した場合、例外メッセージを格納
                df.at[j, "clap_text_embed"] = "ERROR"
                print(str(e))
                e_num = e_num +1

        # 結果を表示
        print(df)
        print(f"ERROR数：{e_num}")

        # CSVファイルに結果を保存
        df.to_csv("../Datasets/archive/musiccaps-processed.csv", index=False)

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# CSVファイル、抽出列を指定
csv_file_path = "../Datasets/archive/musiccaps-processed.csv"
target_column_name = "aspect_list"
#target_column_name = "caption" # caption_clip用

clap_process_and_add_embedding(csv_file_path, target_column_name)


             ytid  start_s  end_s  \
0     -0Gj8-vB1q4       30     40   
1     -0SdAVK79lg       30     40   
2     -0vPFx-wRRI       30     40   
3     -0xzrMun0Rs       30     40   
4     -1LrH01Ei1w       30     40   
...           ...      ...    ...   
5516  zw5dkiklbhE       15     25   
5517  zwfo7wnXdjs       30     40   
5518  zx_vcwOsDO4       50     60   
5519  zyXa2tdBTGc       30     40   
5520  zzNdwF40ID8       70     80   

                               audioset_positive_labels  \
0                          /m/0140xf,/m/02cjck,/m/04rlf   
1     /m/0155w,/m/01lyv,/m/0342h,/m/042v_gx,/m/04rlf...   
2                                   /m/025_jnm,/m/04rlf   
3                                    /m/01g90h,/m/04rlf   
4                                   /m/02p0sh1,/m/04rlf   
...                                                 ...   
5516                                /m/01sm1g,/m/0l14md   
5517                      /m/02p0sh1,/m/04rlf,/m/06j64v   
5518  /m/01glhc,/m/02sgy

In [ ]:
import torch
df = pd.read_csv(csv_file_path)
text_data_list = df["clap_text_embed"].tolist()
tensor_embed = torch.tensor(eval(text_data_list[0]))
print(tensor_embed)

In [ ]:
# Try to recreate Tensor from CSV`s string object
import pandas as pd
import torch
import ast

def load_csv_and_convert_to_tensor(csv_path):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # "clap_text_embed" 列のデータをテンソル形式に変換して格納するリスト
        tensor_list = []

        # "clap_text_embed" 列のデータを順に処理してテンソルに変換
        for text_embed_str in df["clap_text_embed"]:
            try:
                # 文字列形式のテンソルをリストに変換
                tensor_list_str = ast.literal_eval(text_embed_str)

                # リストをPyTorchのテンソルに変換
                text_embed_tensor = torch.tensor(tensor_list_str)

                # テンソルをリストに追加
                tensor_list.append(text_embed_tensor)

            except Exception as e:
                # エラーが発生した場合、Noneを追加しておく
                tensor_list.append(None)
                print(str(e))

        # 結果を表示
        for tensor in tensor_list:
            print(tensor)

        # テンソル形式のリストを返す
        return tensor_list

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return None

# CSVファイルのパスを指定
csv_file_path = "../Datasets/archive/musiccaps-processed.csv"

# CSVファイルからテンソル形式のリストを読み込む
tensor_list = load_csv_and_convert_to_tensor(csv_file_path)


In [ ]:
text_data_list = df["clip_text_embed"].tolist()
print(text_data_list[0])

**CLAP** textdata-processing ->save to .npy

In [ ]:
import pandas as pd
import torch
import numpy as np
import pickle

def clip_process_and_add_embedding(csv_path, column_name, save_path):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clap_text_embed" を追加
        df["clip_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # NumPy配列で結果を保存するためのリスト
        embedding_array_list = []

        # テキストデータを順に処理して "clap_text_embed" に格納
        for text_data in text_data_list:
            try:
                # text_dataを2要素の配列に変換
                text_data_array = [text_data, ""]
                
                with torch.no_grad():
                    # model2.get_text_embeddingの前にtext_dataを配列に変換
                    text_embed = model2.get_text_embedding(text_data_array, use_tensor=True).to(device)

                    # [2, 512]のTensorから第一要素を取り出し、[1, 512]のTensorにする
                    text_embed = text_embed[0].unsqueeze(0)

                    # CUDAデバイス上のテンソルをCPU上のNumPy配列に変換
                    text_embed_np = text_embed.cpu().numpy()

                # テンソルをNumPy配列に変換してリストに追加
                embedding_array_list.append(text_embed_np)

            except Exception as e:
                # エラーが発生した場合、エラー内容を表示して保存する
                error_message = str(e)
                embedding_array_list.append(np.array([float('nan')] * 512))  # エラーの場合はNaNで埋める
                print(f"エラーが発生しました: {error_message}")

        # NumPy配列のリストを結合して、2次元のNumPy配列に変換
        embedding_array = np.vstack(embedding_array_list)

        # NumPy配列を保存
        with open(save_path, 'wb') as f:
            pickle.dump(embedding_array, f)

        # 結果を表示
        print("結果が保存されました。")

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# CSVファイル、抽出列、保存パスを指定
csv_file_path = "../Datasets/archive/musiccaps-processed.csv"
target_column_name = "aspect_list"
numpy_save_path = "../Datasets/archive/clip_embeds.npy"

clip_process_and_add_embedding(csv_file_path, target_column_name, numpy_save_path)


**CLAP** textdata-processing ->save to .npy

In [ ]:
import pandas as pd
import torch
import numpy as np
import pickle

def clap_process_and_add_embedding(csv_path, column_name, save_path):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clap_text_embed" を追加
        df["clap_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # NumPy配列で結果を保存するためのリスト
        embedding_array_list = []

        # テキストデータを順に処理して "clap_text_embed" に格納
        for text_data in text_data_list:
            try:
                # text_dataを2要素の配列に変換
                text_data_array = [text_data, ""]
                
                with torch.no_grad():
                    # model2.get_text_embeddingの前にtext_dataを配列に変換
                    text_embed = model2.get_text_embedding(text_data_array, use_tensor=True).to(device)

                    # [2, 512]のTensorから第一要素を取り出し、[1, 512]のTensorにする
                    text_embed = text_embed[0].unsqueeze(0)

                    # CUDAデバイス上のテンソルをCPU上のNumPy配列に変換
                    text_embed_np = text_embed.cpu().numpy()

                # テンソルをNumPy配列に変換してリストに追加
                embedding_array_list.append(text_embed_np)

            except Exception as e:
                # エラーが発生した場合、エラー内容を表示して保存する
                error_message = str(e)
                embedding_array_list.append(np.array([float('nan')] * 512))  # エラーの場合はNaNで埋める
                print(f"エラーが発生しました: {error_message}")

        # NumPy配列のリストを結合して、2次元のNumPy配列に変換
        embedding_array = np.vstack(embedding_array_list)

        # NumPy配列を保存
        with open(save_path, 'wb') as f:
            pickle.dump(embedding_array, f)

        # 結果を表示
        print("結果が保存されました。")

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# CSVファイル、抽出列、保存パスを指定
csv_file_path = "../Datasets/archive/musiccaps-processed.csv"
target_column_name = "aspect_list"
numpy_save_path = "../Datasets/archive/clap_embeds.npy"

clap_process_and_add_embedding(csv_file_path, target_column_name, numpy_save_path)


In [ ]:
import torch
import numpy as np

# npyファイルからNumPy配列を読み込む
numpy_array = np.load(numpy_save_path, allow_pickle=True)

# NumPy配列をPyTorchのテンソルに変換する
tensor = torch.tensor(numpy_array)

# 最初の要素を表示する
print("最初の要素:", tensor[0])

In [ ]:
print(tensor[0].shape)
print(tensor[0].unsqueeze(0))
print(tensor[0].unsqueeze(0).shape)

**CLIP**&**CLAP** textdata-processing ->save to .npy

In [ ]:
import pandas as pd
import torch
import numpy as np
import pickle

def clap_process_and_add_embedding(csv_path, column_name, save_path):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clap_text_embed_A" と "clap_text_embed_B" を追加
        df["clip_text_embed"] = None
        df["clap_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # NumPy配列で結果を保存するためのリスト（AとBの処理結果用）
        embedding_array_list_clip = []
        embedding_array_list_clap = []

        # テキストデータを順に処理して "clap_text_embed" に格納
        for text_data in text_data_list:
            try:
                # text_dataを2要素の配列に変換
                text_data_array_clap = [text_data, ""]
                
                with torch.no_grad():
                    # model2.get_text_embeddingの前にtext_dataを配列に変換
                    text_embed_A = model2.get_text_embedding(text_data_array, use_tensor=True).to(device)
                    text_embed_B = model3.get_text_embedding(text_data_array, use_tensor=True).to(device)

                    # [2, 512]のTensorから第一要素を取り出し、[1, 512]のTensorにする
                    text_embed_A = text_embed_A[0].unsqueeze(0)
                    text_embed_B = text_embed_B[0].unsqueeze(0)

                    # CUDAデバイス上のテンソルをCPU上のNumPy配列に変換
                    text_embed_np_A = text_embed_A.cpu().numpy()
                    text_embed_np_B = text_embed_B.cpu().numpy()

                # テンソルをNumPy配列に変換してリストに追加
                embedding_array_list_A.append(text_embed_np_A)
                embedding_array_list_B.append(text_embed_np_B)

            except Exception as e:
                # エラーが発生した場合、エラー内容を表示して保存する
                error_message = str(e)
                embedding_array_list_A.append(np.array([float('nan')] * 512))  # エラーの場合はNaNで埋める
                embedding_array_list_B.append(np.array([float('nan')] * 512))  # エラーの場合はNaNで埋める
                print(f"エラーが発生しました: {error_message}")

        # NumPy配列のリストを結合して、2次元のNumPy配列に変換
        embedding_array_A = np.vstack(embedding_array_list_A)
        embedding_array_B = np.vstack(embedding_array_list_B)

        # NumPy配列を保存
        with open(save_path + "_A.npy", 'wb') as f:
            np.save(f, embedding_array_A)
        with open(save_path + "_B.npy", 'wb') as f:
            np.save(f, embedding_array_B)

        # 結果を表示
        print("結果が保存されました。")

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# CSVファイル、抽出列、保存パスを指定
csv_file_path = "../Datasets/archive/musiccaps-processed.csv"
target_column_name = "aspect_list"
numpy_save_path = "../Datasets/archive/clap_embeds"

clap_process_and_add_embedding(csv_file_path, target_column_name, numpy_save_path)
